## 01-running-cool-roof-process
Rachel Bowers, Analayst II at Metropolitan Area Planning Council


This notebook runs scripts to process Lidar point cloud data, roofprint data, and assessors data from MassGIS, resulting in building footprints for all of Metro Mayors region with added fields for:
- Whether the average pitch of the roof indicates that it is a "low slope" roof suitable for cool roof conversion
- The relative reflectivity over the roofprint area
- Additional land use and socio-environmental variables that indicate whether cool roof conversions may be particularly impactful or appropriate

In [ ]:
#read in scripts  
%load_ext autoreload
%autoreload 2



import sys
sys.path.append("..")

from src.features.create_rasters import *#
#from src.features.arcpy_scripts import *
from src.features.custom_functions import * 


### Run these only when new lidar or to re-run heat_score_mmc

- las_dataset was last created for all MMC on 5/1/2024
- heat_score_mmc was last created for all MMC on 5/1/2024

In [ ]:
#only run this if need be [last run on all MMC: 5/1/2024]
from src.data.make_dataset import heat_fp, las_folder

#dataset file path
las_dataset = r'I:\Imagery\MassGIS_LAS_files\MMC_lasdataset.lasd'

las_dataset = create_las_dataset(las_folder=las_folder, las_dataset=las_dataset)

heat_score_mmc = get_heat_score_mmc(heat_index_fp=heat_fp)

### First create slope and intensity rasters for each municipality

Result in geodatabase should be just a slope and intensity raster for each muni - note, as of 5/22, this can be skipped (unless you need to rerun any of them)

In [ ]:
mmc_town_names = ['Arlington', 'Boston', 'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
             'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
             'Somerville', 'Watertown', 'Winthrop']


#note that Revere was removed from this list since it was the test one 

from datetime import datetime

for town_name in mmc_town_names:
    
    print(town_name + ' processing starting at ' + str(datetime.now()))


    #create las dataset 
    las_dataset = create_las_dataset(town_name = town_name, 
                                     las_folder=las_folder) 
    
    #create an ndsm raster
    ndsm_raster = create_ndsm_raster(town_name=town_name,
                                    las_dataset=las_dataset)
    
    #create a slope raster that covers each roofprint
    slope_raster = create_slope_raster(town_name=town_name,
                                        ndsm_raster=ndsm_raster)
    
    #removing aspect for now, may include later to refine the flat roof analysis
    #aspect_raster = create_aspect_raster(town_name=town_name,
    #                                     ndsm_raster=ndsm_raster)

    #create an intensity raster that covers each roofprint
    intensity_raster = create_intensity_raster(town_name=town_name, 
                                                las_dataset=las_dataset)


### Then create enriched roofprint polygon datasets

Enriches roofprint data with land use and socio-enviro fields

In [ ]:
from src.data.make_dataset import mmc_heat_blocks
from datetime import datetime


mmc_town_names = ['Arlington', 'Boston', 'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
             'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
             'Somerville', 'Watertown', 'Winthrop']

for town_name in mmc_town_names:
  
    data_folder = r'K:\DataServices\Projects\Current_Projects\Climate_Change\MVP_MMC_CoolRoofs_MVP\Data\Intermediate'
    cool_roof_footprints_layer = os.path.join(data_folder, (town_name + '_cool_roofs.shp'))
    
    if arcpy.Exists(cool_roof_footprints_layer): #an existing out_slope_raster would mean this town has already been processed
        print('cool roofs have been processed for ' + town_name + '. Moving to next town.')
        pass


    else:
        print(town_name + ' processing starting at ' + str(datetime.now()))
    
        rooftops_layer = make_coolroof_roofprints_layer(town_name=town_name)
        
        #input for the cool_roof_process() function has to be a gdf
        rooftops_gdf = gpd.read_file(cool_roofs_gdb, layer=(town_name + '_footprints'))

        town_structures_w_info = cool_roof_process(town_name=town_name, 
                                                    rooftops_layer=rooftops_gdf,
                                                    heat_blocks_layer=mmc_heat_blocks)


### Optional debugging

To rerun the cool roof processing (rather than the development of the baseline footprint layer)

In [ ]:
#rerunning just the cool_roof_process() part

from datetime import datetime

from src.data.make_dataset import mmc_heat_blocks

mmc_town_names = ['Arlington', 'Boston', 'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
                'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
                'Somerville', 'Watertown', 'Winthrop']

for town_name in mmc_town_names:
    print(town_name, ' starting at ', datetime.now())

    rooftops_gdf = gpd.read_file(cool_roofs_gdb, layer=(town_name + '_footprints'))

    town_structures_w_info = cool_roof_process(town_name=town_name, 
                                                rooftops_layer=rooftops_gdf,
                                                heat_blocks_layer=mmc_heat_blocks)


### Merge shapefiles together

In [ ]:
## MERGE SHAPEFILES ##

from src.data.make_dataset import input_dir, intermediate_path, output_dir

filelist = []

# add all shapefiles
for root, folder, files in os.walk(intermediate_path):
    for file in files:
        for muni in mmc_town_names:
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)

merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)

#export
output_path = os.path.join(output_dir, 'MMC_Cool_Roofs.shp')
merged_gdf.to_file(output_path)

### Optional debugging: Replace any towns as needed 

In [ ]:
#replace with any towns you want to rerun analysis on
munis_to_fix = ['Watertown']

## RUN ANALYSIS ON MUNIS OF CHOICE ## 

from src.data.make_dataset import intermediate_path, output_dir


# make cool roof roofprints, add socio-enviro fields

for town_name in munis_to_fix:
    print('rerunning analysis on ' + town_name)

    rooftops_layer = make_coolroof_roofprints_layer(town_name=town_name)
    
    # input for the cool_roof_process() function has to be a gdf
    rooftops_gdf = gpd.read_file(cool_roofs_gdb, layer=(town_name + '_footprints'))

    town_structures_w_info = cool_roof_process(town_name=town_name, 
                                                rooftops_layer=rooftops_gdf,
                                                heat_blocks_layer=mmc_heat_blocks)


# output of the above is a shapefile to be read back in

for root, folder, files in os.walk(intermediate_path):
    for file in files:
        for town_name in munis_to_fix:
            if town_name in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


fixed_munis_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)


# read in existing gdf of all MMC munis 
print('reading in existing dataset')
output_path = os.path.join(output_dir, 'MMC_Cool_Roofs.shp')
existing_gdf = gpd.read_file(output_path)

# create new gdf that excludes the existing rows from the munis of interest
existing_gdf_minus_munis = existing_gdf.loc[~existing_gdf.CITY.isin(munis_to_fix)]

# add the gdf with fixed munis
fixed_gdf = pd.concat([existing_gdf_minus_munis, fixed_munis_gdf])
print('exporting final dataset')
fixed_gdf.to_file(output_path)